## 컨텐츠 기반 필터링(Content-based Filtering)

In [18]:
import numpy as np
import pandas as pd

In [19]:
from surprise import Dataset
data = Dataset.load_builtin('ml-100k', prompt=False)
df = pd.DataFrame(data.raw_ratings, columns=['user-id', 'movie-id', 'rating', 'timestamp'])
df.head()

,user-id,movie-id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [20]:
df.shape

(100000, 4)

### 1. Adjacent Matrix 생성
- 행은 사용자
- 열은 영화
- 내용은 평점

In [21]:
raw_data = np.array(data.raw_ratings, dtype=int)
np.min(raw_data, axis=0)

array([        1,         1,         1, 874724710])

In [22]:
np.max(raw_data, axis=0)

array([      943,      1682,         5, 893286638])

In [23]:
# user-id, movie-id가 0부터 시작하도록 만들어 줌
raw_data[:,:2] -= 1
raw_data

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       ...,
       [      275,      1089,         1, 874795795],
       [       12,       224,         2, 882399156],
       [       11,       202,         3, 879959583]])

In [24]:
# 행, 열의 개수
nrows = df['user-id'].nunique()     # 행의 개수 = 사용자 수
ncols = df['movie-id'].nunique()    # 열의 개수 = 영화 수
nrows, ncols

(943, 1682)

#### 1) 본 영화 / 안 본 영화 로만 구분, 0/1

In [25]:
# 본 영화: 1/ 안 본 영화: 0
adj_metrix = np.zeros([nrows, ncols], int)
for user_id, movie_id, _, _ in raw_data:
    adj_metrix[user_id, movie_id] = 1
adj_metrix[:5]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [26]:
# 0번 데이터를 나라고 함
my_id, my_vector = 0, adj_metrix[0]

In [27]:
# 유사도 - 이진 벡터의 내적
# 나와 10,20번 사용자와의 유사도
np.dot(my_vector, adj_metrix[10]), np.dot(my_vector, adj_metrix[20])

(71, 42)

In [28]:
# 누가 나랑 닮았나?
best_score = 0; best_match_id = 0

for i in range(1, len(adj_metrix)):
    dot = np.dot(my_vector, adj_metrix[i])
    if dot > best_score:
        best_score, best_match_id = dot, i
best_score, best_match_id

(183, 275)

In [29]:
best_vector = adj_metrix[best_match_id]
my_vector[100:110], best_vector[100:110]

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), array([1, 0, 0, 1, 0, 0, 0, 1, 1, 0]))

In [30]:
# 내가 보지 않은 영화중에서 best_match 사용자가 본 영화 --> 추천
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_vector)):
    if my_view == 0 and best_match_view == 1:
        recommend_list.append(i)

len(recommend_list), recommend_list[:10]

(335, [272, 273, 275, 280, 281, 283, 287, 288, 289, 290])

#### 2) 평점 점수를 주는 경우

In [31]:
adj_metrix = np.zeros([nrows, ncols], int)
for user_id, movie_id, rating, _ in raw_data:
    adj_metrix[user_id, movie_id] = rating
adj_metrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

- Case 1: 유클리드 거리

In [32]:
# 누가 나랑 닮았나?
best_score, best_match_id = 100000, 0
my_vector = adj_metrix[0]

for i in range(1, len(adj_metrix)):
    euc = np.sqrt(np.sum(np.square(my_vector - adj_metrix[i])))     # (np.square(my_vector - adj_metrix[i]))
    if euc < best_score:
        best_score, best_match_id = euc, i

best_score, best_match_id

(55.06359959174482, 737)

In [33]:
# 내가 보지 않은 영화 중에서 best_match 사용자가 본 영화 --> 추천
best_vector = adj_metrix[best_match_id]
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_vector)):
    if my_view == 0 and best_match_view >= 1:
        recommend_list.append(i)

len(recommend_list), recommend_list[:10]

(48, [297, 312, 317, 342, 356, 366, 379, 384, 392, 402])

In [36]:
# 내가 보지 않은 영화중에서 best_match 사용자가 본 평점이 좋은 영화 --> 추천
best_vector = adj_metrix[best_match_id]
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_vector)):
    if my_view == 0 and best_match_view >= 4:
        recommend_list.append(i)

len(recommend_list), recommend_list[:10]

(21, [312, 317, 356, 384, 407, 422, 433, 454, 469, 473])

- Case 2: 코사인 유사도

In [37]:
def cos_similarity(v1, v2):
    v1_norm = np.sqrt(np.sum(np.square(v1)))
    v2_norm = np.sqrt(np.sum(np.square(v2)))
    return np.dot(v1, v2) / (v1_norm * v2_norm)

In [42]:
# 누가 나랑 닮았나?
best_score, best_match_id = -1, 0
my_vector = adj_metrix[0]

for i in range(1, len(adj_metrix)):
    cos_sim = cos_similarity(my_vector, adj_metrix[i])
    if cos_sim > best_score:
        best_score, best_match_id = cos_sim, i

best_score, best_match_id

(0.569065731527988, 915)

In [43]:
# 내가 보지 않은 영화중에서 best_match 사용자가 본 평점이 좋은 영화 --> 추천
best_vector = adj_metrix[best_match_id]
recommend_list = []
for i, (my_view, best_match_view) in enumerate(zip(my_vector, best_vector)):
    if my_view == 0 and best_match_view >= 4:
        recommend_list.append(i)

len(recommend_list), recommend_list[:10]

(58, [275, 285, 316, 317, 381, 386, 420, 424, 426, 427])